In [1]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import jp_proxy_widget as jp

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
def getdata(ICAO):
    apiKey = "gQvOAvCu725jH4VcgCA300hbQJNwFfz2"
    apiUrl = "https://aeroapi.flightaware.com/aeroapi/"

    operators = ICAO
    payload = {'max_pages': 4}
    auth_header = {'x-apikey':apiKey}

    response = requests.get(apiUrl + f"operators/{operators}/flights",
    params=payload, headers=auth_header)

    if response.status_code == 200:
        output = response.json()
        return output
    else:
        return("Error executing request")

def getflight(callsign):
    apiKey = "gQvOAvCu725jH4VcgCA300hbQJNwFfz2"
    apiUrl = "https://aeroapi.flightaware.com/aeroapi/"

    flight = str(callsign)
    payload = {'max_pages': 4}
    auth_header = {'x-apikey':apiKey}

    response = requests.get(apiUrl + f"flights/{flight}",
        params=payload, headers=auth_header)

    if response.status_code == 200:
        output = response.json()
        return output
    else:
        return("Error executing request")


In [2]:
flightdata = getflight(input('Input flight callsign. e.g."UAL1"'))
flight = pd.json_normalize(flightdata['flights'])
airline = flight['operator_icao'][0]
departure_delay = flight['departure_delay'][0]
aircraft = flight['aircraft_type'][0]
dist = flight['route_distance'][0]
origin = flight['origin.code_icao'][0]
destination = flight['destination.code_icao'][0]
print(origin)

KSFO


In [3]:
airline = str(airline)
output = getdata(airline)
scheduled = pd.json_normalize(output['scheduled'])
arrivals = pd.json_normalize(output['arrivals'])
arrivals_cleaned=arrivals[["ident_icao","operator","departure_delay","arrival_delay","aircraft_type","route_distance","origin.code_icao","destination.code_icao"]]
X = arrivals_cleaned[["operator","departure_delay","aircraft_type","route_distance","origin.code_icao","destination.code_icao"]]
dataset = X.copy()
y = arrivals_cleaned['arrival_delay']
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X['operator'])
sequences = tokenizer.texts_to_sequences(X['operator'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['operator'] = sequences
tokenizer.fit_on_texts(X['aircraft_type'])
sequences = tokenizer.texts_to_sequences(X['aircraft_type'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['aircraft_type'] = sequences
tokenizer.fit_on_texts(X['origin.code_icao'])
sequences = tokenizer.texts_to_sequences(X['origin.code_icao'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['origin.code_icao'] = sequences
tokenizer.fit_on_texts(X['destination.code_icao'])
sequences = tokenizer.texts_to_sequences(X['destination.code_icao'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['destination.code_icao'] = sequences
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=6))  # Input layer with 6 features
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='linear'))  # Output layer for regression
model.compile(optimizer='adam', loss='mean_squared_error')  # Use an appropriate loss function for your task
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test))
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
y_pred = model.predict(X_test)


Epoch 1/1000


/var/folders/52/14hdcd_d3451gmf6wbwp_lsc0000gn/T/ipykernel_33359/2443182731.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['operator'] = sequences
/var/folders/52/14hdcd_d3451gmf6wbwp_lsc0000gn/T/ipykernel_33359/2443182731.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['aircraft_type'] = sequences
/var/folders/52/14hdcd_d3451gmf6wbwp_lsc0000gn/T/ipykernel_33359/2443182731.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

2/2 [==============================] - 0s 66ms/step - loss: 19935326.0000 - val_loss: 2002120.5000
Epoch 2/1000
2/2 [==============================] - 0s 12ms/step - loss: 19935130.0000 - val_loss: 2002048.0000
Epoch 3/1000
2/2 [==============================] - 0s 13ms/step - loss: 19934960.0000 - val_loss: 2001976.3750
Epoch 4/1000
2/2 [==============================] - 0s 12ms/step - loss: 19934778.0000 - val_loss: 2001916.0000
Epoch 5/1000
2/2 [==============================] - 0s 13ms/step - loss: 19934612.0000 - val_loss: 2001851.3750
Epoch 6/1000
2/2 [==============================] - 0s 13ms/step - loss: 19934482.0000 - val_loss: 2001790.0000
Epoch 7/1000
2/2 [==============================] - 0s 13ms/step - loss: 19934266.0000 - val_loss: 2001732.3750
Epoch 8/1000
2/2 [==============================] - 0s 13ms/step - loss: 19934144.0000 - val_loss: 2001672.5000
Epoch 9/1000
2/2 [==============================] - 0s 13ms/step - loss: 19933984.0000 - val_loss: 2001613.5000
Epoch

In [4]:
print(y_pred)

[[-3804.883   ]
 [-1602.2723  ]
 [-2477.5627  ]
 [ -632.1613  ]
 [ 1424.07    ]
 [ -126.258385]
 [-1469.419   ]
 [-2194.4082  ]
 [-2054.36    ]
 [-1011.40405 ]
 [-2176.7021  ]
 [  912.53174 ]]


In [5]:
print(y_test)

0    -1500
5     -360
36   -1421
45   -2460
13   -1140
54    1080
33   -1740
48    -900
12    -960
57    -480
46    -960
50    2280
Name: arrival_delay, dtype: int64


In [6]:
print(X_test)

[[ 0.         -0.3396866   0.46003574 -0.41106017 -0.52905209 -1.19661129]
 [ 0.         -0.3396866  -0.07854269 -1.63857324  2.41727114  0.92333674]
 [ 0.         -0.37467578 -0.88641033  0.1384908  -0.66935319  1.38707537]
 [ 0.         -0.37467578  1.26790339  0.18352598 -0.66935319  1.65206887]
 [ 0.         -0.3396866  -1.15569955  1.24550432  1.29486229 -0.20288565]
 [ 0.         -0.16474071 -0.61712112  0.68439026 -0.52905209  1.850814  ]
 [ 0.         -0.6545892   0.99861417 -1.68482559 -0.38875098  1.25457862]
 [ 0.         -0.40966496  0.72932496 -0.44818377 -0.66935319 -0.46787915]
 [ 0.         -0.44465413 -0.07854269 -0.82063691  1.85606671 -0.40163078]
 [ 0.         -0.58461084  0.99861417 -0.74517255  0.59335676  1.98331075]
 [ 0.         -0.58461084 -0.07854269 -0.82246266  1.85606671 -0.40163078]
 [ 0.          1.09486968  0.99861417 -1.17909264  1.99636782  0.45959811]]


In [7]:
flight_requested = pd.DataFrame({'operator':[airline],'departure_delay':[departure_delay],'aircraft_type':[aircraft],'route_distance':[dist],'origin.code_icao':[origin],'destination.code_icao':[destination]})
print(flight_requested)


  operator  departure_delay aircraft_type  route_distance origin.code_icao  \
0      UAL                0           787            8448             KSFO   

  destination.code_icao  
0                  WSSS  
